In [1]:
from huggingface_hub import notebook_login, interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to C:\Users\katko\.cache\huggingface\token
Login successful


In [2]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

In [75]:
task = "srl" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [48]:
from preprocessing import extract_data_from_conll_extended
extract_data_from_conll_extended("../data/en_ewt-up-dummy.conllu", "dummy_test.json")

In [49]:
from datasets import load_dataset, load_metric
datasets = load_dataset('json', data_files={'train': 'dummy_train.json', 'test': 'dummy_test.json'})
datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['predicate', 'predicate_position', 'arguments', 'word'],
        num_rows: 12
    })
    test: Dataset({
        features: ['predicate', 'predicate_position', 'arguments', 'word'],
        num_rows: 12
    })
})

In [50]:
datasets["train"][0]

{'predicate': 'kill.01',
 'predicate_position': 7,
 'arguments': ['_',
  '_',
  '_',
  '_',
  '_',
  'ARG0',
  'V',
  'ARG1',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  'ARGM-LOC',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_'],
 'word': ['Al',
  '-',
  'Zaman',
  ':',
  'American',
  'forces',
  'killed',
  'Shaikh',
  'Abdullah',
  'al',
  '-',
  'Ani',
  ',',
  'the',
  'preacher',
  'at',
  'the',
  'mosque',
  'in',
  'the',
  'town',
  'of',
  'Qaim',
  ',',
  'near',
  'the',
  'Syrian',
  'border',
  '.']}

In [51]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [52]:
show_random_elements(datasets["train"])

,predicate,predicate_position,arguments,word
0,operate.01,14,"[_, _, _, _, _, _, _, _, _, _, _, _, ARG0, V, _, ARGM-LOC, _]","[DPA, :, Iraqi, authorities, announced, that, they, had, busted, up, 3, terrorist, cells, operating, in, Baghdad, .]"
1,run.01,6,"[ARG1, _, _, _, _, V, _, _, ARG0, _, _, _, _, _, _, _]","[Two, of, them, were, being, run, by, 2, officials, of, the, Ministry, of, the, Interior, !]"
2,kill.01,3,"[_, _, V, _, _, _, ARG1, _, _, _, _, _, _, _, _, _, _, _]","[[, This, killing, of, a, respected, cleric, will, be, causing, us, trouble, for, years, to, come, ., ]]"
3,be.03,9,"[_, _, _, _, _, _, _, _, V, _, _, _, _, _, _, _, _, _]","[[, This, killing, of, a, respected, cleric, will, be, causing, us, trouble, for, years, to, come, ., ]]"
4,have.01,8,"[_, _, _, _, _, _, _, V, _, _, _, _, _, _, _, _, _]","[DPA, :, Iraqi, authorities, announced, that, they, had, busted, up, 3, terrorist, cells, operating, in, Baghdad, .]"
5,be.03,4,"[_, _, _, V, _, _, _, _, _, _, _, _, _, _, _, _]","[Two, of, them, were, being, run, by, 2, officials, of, the, Ministry, of, the, Interior, !]"
6,kill.01,7,"[_, _, _, _, _, ARG0, V, ARG1, _, _, _, _, _, _, _, _, _, ARGM-LOC, _, _, _, _, _, _, _, _, _, _, _]","[Al, -, Zaman, :, American, forces, killed, Shaikh, Abdullah, al, -, Ani, ,, the, preacher, at, the, mosque, in, the, town, of, Qaim, ,, near, the, Syrian, border, .]"
7,bust_up.04,9,"[_, _, _, _, _, _, ARG0, _, V, _, _, _, ARG1, _, _, _, _]","[DPA, :, Iraqi, authorities, announced, that, they, had, busted, up, 3, terrorist, cells, operating, in, Baghdad, .]"
8,announce.01,5,"[_, _, _, ARG0, V, _, _, _, ARG1, _, _, _, _, _, _, _, _]","[DPA, :, Iraqi, authorities, announced, that, they, had, busted, up, 3, terrorist, cells, operating, in, Baghdad, .]"
9,cause.01,10,"[_, _, ARG0, _, _, _, _, ARGM-MOD, _, V, ARGM-GOL, ARG1, _, ARGM-TMP, _, _, _, _]","[[, This, killing, of, a, respected, cleric, will, be, causing, us, trouble, for, years, to, come, ., ]]"


In [67]:
datasets["train"][0]

{'predicate': 'kill.01',
 'predicate_position': 7,
 'arguments': ['_',
  '_',
  '_',
  '_',
  '_',
  'ARG0',
  'V',
  'ARG1',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  'ARGM-LOC',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_'],
 'word': ['Al',
  '-',
  'Zaman',
  ':',
  'American',
  'forces',
  'killed',
  'Shaikh',
  'Abdullah',
  'al',
  '-',
  'Ani',
  ',',
  'the',
  'preacher',
  'at',
  'the',
  'mosque',
  'in',
  'the',
  'town',
  'of',
  'Qaim',
  ',',
  'near',
  'the',
  'Syrian',
  'border',
  '.']}

In [53]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [54]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [65]:
def preprocess(instance):
    sentence = instance["word"]
    tokenized_input = tokenizer(sentence, is_split_into_words=True)
    predicate = instance['predicate']
    predicate = tokenizer(predicate)
    tokenized_input['input_ids'].extend(predicate['input_ids'][1:]), tokenized_input['attention_mask'].extend(predicate['attention_mask'][1:])
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    return tokenized_input, tokens

In [69]:
training_set = [{'x': preprocess(instance)[0], 'y': instance['arguments']} for instance in datasets['train']]
training_set[0]

{'x': {'input_ids': [101, 2632, 1011, 23564, 2386, 1024, 2137, 2749, 2730, 21146, 28209, 14093, 2632, 1011, 2019, 2072, 1010, 1996, 14512, 2012, 1996, 8806, 1999, 1996, 2237, 1997, 1053, 4886, 2213, 1010, 2379, 1996, 9042, 3675, 1012, 102, 3102, 1012, 5890, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 'y': ['_',
  '_',
  '_',
  '_',
  '_',
  'ARG0',
  'V',
  'ARG1',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  'ARGM-LOC',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_',
  '_']}

In [57]:
print(tokenized_input.word_ids())


[None, 0, 1, 2, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 22, 22, 23, 24, 25, 26, 27, 28, None]


In [77]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
label_list = set(label for instance in datasets["train"] for label in instance['arguments'])

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [71]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [72]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [73]:
metric = load_metric("seqeval")
metric = load_metric("seqeval")


C:\Users\katko\miniconda3\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
C:\Users\katko\miniconda3\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metri

In [76]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

KeyError: 'srl_tags'